In [119]:
from sqlalchemy import create_engine

# Your ODBC connection string
odbc_str = 'DRIVER={ODBC Driver 18 for SQL Server};SERVER=DESKTOP-HM7LQLF\\VAIBHAVSERVER;DATABASE=cricllm;Trusted_Connection=yes;TrustServerCertificate=yes;'

# Create SQLAlchemy engine
db_engine = create_engine(f'mssql+pyodbc:///?odbc_connect={odbc_str}')


In [104]:
import os
from dotenv import load_dotenv

os.environ["OPENAI_API_TYPE"]="azure"
os.environ["OPENAI_API_VERSION"]="2023-07-01-preview"
os.environ["OPENAI_API_BASE"]="https://cricai.openai.azure.com//openai/deployments/cricaidep/chat/completions?api-version=2023-07-01" # Your Azure OpenAI resource endpoint
os.environ["OPENAI_API_KEY"]="905cac03e827471192d7d24351bf6374" # Your Azure OpenAI resource key
os.environ["OPENAI_CHAT_MODEL"]="gpt-35-turbo-16k" # Use name of deployment

In [105]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(model=os.getenv("OPENAI_CHAT_MODEL"),
                      deployment_name=os.getenv("OPENAI_CHAT_MODEL"),
                      temperature=0)

C:\Users\User\anaconda3\envs\CricLLM\lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\User\anaconda3\envs\CricLLM\lib\site-packages\langchain_community\chat_models\azure_openai.py:182: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/deployments/example-deployment. Updating https://cricai.openai.azure.com//openai/deployments/cricaidep/chat/completions?api-version=2023-07-01 to https://cricai.openai.azure.com//openai/deployments/cricaidep/chat/completions?api-version=2023-07-01.
  warnings.warn(


In [122]:
from langchain.prompts.chat import ChatPromptTemplate

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
          You are a helpful AI assistant expert in querying SQL Database to find answers to user's question about Matches
          ID: A unique identifier for each cricket match, column type - int
City: The city where the match took place, providing geographical context for the event. column type - varchar
Date: The date of the match.
Season: The season year of that match.
MatchNumber: The match number within the season and also describes type of matches like Final,
Qualifier 2,
Eliminator,
Qualifier 1,if match is of playoffs type.
Team1: The name of the first team playing in the match. column type - varchar
Team2: The name of the second team playing in the match. column type - varchar
Venue: The name of the stadium where the match was held. column type - varchar
TossWinner: The name of team winning the toss. column type - varchar
TossDecision: The decision made by the toss-winning team which can be bat or field. column type - varchar
SuperOver: Whether a Super Over was played to break a tie, Y if played , N if super over is not player. 
WinningTeam: The name of team that won the that match. column type - varchar
WonBy: Describes how the match was won by Runs,Wickets, SuperOver or NoResult. column type - varchar
Margin: The margin of victory, if corresponding WonBy cell is "Runs" then no. of runs would mentioned, if corresponding Wonby cell is "Wickets" then no. of wickets by which winning team won would be monetioned. column type - int
Method: The method of winning either D/L or NA , if there is rain then D/L method is used , otherwise NA which describes normal match. column type - varchar
Player_of_Match: The name of Man of the Match player. column type - varchar 

Team1Players: A list of players in Team 1, providing information about the team composition. Example - ["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8","Player9","Player10","Player11"]

Team2Players: A list of players in Team 2, similar example as Team1Players column

Umpire1: The name of first umpire officiating the match. column type - varchar

Umpire2: The name of second umpire officiating the match. column type - varchar

         """
         ),
        ("user", "{question}\n ai: "),
    ]
)

In [123]:
from langchain.agents import AgentType, create_sql_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

db = SQLDatabase(db_engine)

sql_toolkit = SQLDatabaseToolkit(db=db, llm=llm)
sql_toolkit.get_tools()

sqldb_agent = create_sql_agent(
    llm=llm,
    toolkit=sql_toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [127]:
sqldb_agent.run(final_prompt.format(
        question="How many matches did Mumbai Indians won by more than 5 wicket margin?"
  ))



> Entering new SQL Agent Executor chain...
We need to query the database to find the number of matches won by Mumbai Indians with a margin of more than 5 wickets.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM Matches WHERE (Team1='Mumbai Indians' OR Team2='Mumbai Indians') AND WonBy='Wickets' AND Margin>5 AND WinningTeam='Mumbai Indians';[(35,)]The query returned a result of [(35,)], which means that Mumbai Indians won 35 matches by a margin of more than 5 wickets.
Final Answer: Mumbai Indians won 35 matches by more than 5 wicket margin.

> Finished chain.


'Mumbai Indians won 35 matches by more than 5 wicket margin.'